In [1]:
%matplotlib notebook
import numpy as np
import pandas
import matplotlib.pyplot as plt
import os

Check if you installed the LePhare package.<br>
Website : http://www.cfht.hawaii.edu/~arnouts/LEPHARE/lephare.html

In [2]:
from pylephare import lephare

# Data

Let's load an example of compatible input data: pylephare/data/data_test.csv<br>
The code expects input data in form of a dataframe with the following column format:

filtername1, filtername1.err, filtername2, filtername2.err, ..., filternameN, filternameN.err, CONTEXT, Z-SPEC, STRING

where filtername{} is a known filter by LePhare with the format instrument.band, for instance sdss.u or ps1.z.<br>
The 'CONTEXT' number defines which filter data will be used in the SED fitting. It is equal to the quadratic sum of the used filtername position numbers. For example, using the filters = ["sdss.u", "sdss.g", "sdss.r", "sdss.i", "sdss.z"] (thereby the position numbers are 0 for "sdss.u" until 4 for "sdss.z") :
- CONTEXT = 31 --> ["u", "g", "r", "i", "z"]
- CONTEXT = 30 --> ["g", "r", "i", "z"]
- CONTEXT = 15 --> ["u", "g", "r", "i"]
- CONTEXT = 25 --> ["u", "i", "z"]

The 'Z-SPEC' is the redshift.<br>
In 'STRING' contain all other informations given by the user, such as the SNeIa name in our example.

In [3]:
from pylephare import io

In [4]:
data = pandas.read_csv(io._PACKAGE_ROOT+"/data/data_test.csv", sep=" ")
data

,sdss.u,sdss.u.err,ps1.g,ps1.g.err,ps1.r,ps1.r.err,ps1.i,ps1.i.err,ps1.z,ps1.z.err,CONTEXT,Z-SPEC,STRING
0,1.227194e-17,4.928681e-18,1.389326e-17,6.902862e-20,8.659294e-18,2.922806e-20,9.346882e-18,4.667556e-20,3.230372e-18,2.169672e-20,31,0.03185,2007aj
1,3.189998e-17,4.026194e-18,7.381098e-17,1.799055e-19,7.192829e-17,9.228955e-20,8.032923e-17,1.650248e-19,6.790159e-17,9.719448e-20,31,0.03509,2002G
2,6.248074e-16,7.136033e-18,1.801086e-15,1.164064e-18,2.002177e-15,4.736618e-19,2.109367e-15,8.014284e-19,1.743491e-15,4.585604e-19,31,0.02824,ASASSN-15la


# Classic LePhare SED fitting

Let's make a python wrapper object of LePhare.

The class builder accept some inputs:
- a configuration file directory (if None is given, take the default one, included in the package),
- an output directory to save every results of the SED fitting (if None is given, create a folder, formated by the time and date, in the lephare package: "lephare/lepharework/pylephare/{}"),
- the unit used for input fluxes data (inhz = True if the flux unit is erg/s/cm2/Hz, False for erg/s/cm2/AA)

In [5]:
CONFIG_FILE = None
DIROUT = io.PATH_PYLEPHAREWORK+"/test_ztf_meeting/"

lp = lephare.LePhare(data=data, configfile=CONFIG_FILE, dirout=DIROUT, inhz=False)

running set_filter_suffix
calling set_zphotlib


/Users/mart1/anaconda3/lib/python3.7/site-packages/pylephare-0.2.2-py3.7.egg/pylephare/configparser.py:18: UserWarning: using default config file
  warnings.warn("using default config file")


You can check the configuration parameters thanks to the following command.<br>
You can choose on the format of the output:
- "dict" return a dictionary,
- "dataframe" return a pandas.DataFrame,
- "array" return a numpy.array,
- "configfile" return a friendly visual identical to the given configuration file.

These configuration parameters have been adapted to the created instance. If you want to know what your original input configuration parameters are, you can get them with:
- "original" (return the original configuration file).

In [6]:
lp.config.get_config("file")

['##############################################################################',
 '#                CREATION OF LIBRARIES FROM SEDs List                        #',
 '# $LEPHAREDIR/source/sedtolib -t (S/Q/G) -c $LEPHAREDIR/config/zphot.para    #',
 '# help : $LEPHAREDIR/source/sedtolib -h (or -help)                           #',
 '##############################################################################',
 '#',
 '#------      STELLAR LIBRARY (ASCII SEDs)',
 'STAR_SED $LEPHAREDIR/sed/STAR/STAR_MOD.list # STAR list (full path)',
 'STAR_FSCALE 3.432E-09 # Arbitrary Flux Scale',
 '# STAR_LIB LIB_STAR_sdssu_ps1g_ps1r_ps1i_ps1z',
 '\t\t\t\t\t\t\t# $LEPHAREWORK/lib_bin',
 '#',
 '#------      QSO LIBRARY (ASCII SEDs) ',
 'QSO_SED $LEPHAREDIR/sed/QSO/QSO_MOD.list # QSO list (full path)',
 'QSO_FSCALE 1 # Arbitrary Flux Scale',
 '# QSO_LIB LIB_QSO_sdssu_ps1g_ps1r_ps1i_ps1z',
 '\t\t\t\t\t\t\t# $LEPHAREWORK/lib_bin',
 '#',
 '#------      GALAXY LIBRARY (ASCII or BINARY SEDs) ',
 'GAL_SED $LE

You can check if the data are now in a LePhare compatible format, regarding the columns and the flux unit (erg/cm2/s/Hz).

In [7]:
lp.data

,sdss.u,sdss.u.err,ps1.g,ps1.g.err,ps1.r,ps1.r.err,ps1.i,ps1.i.err,ps1.z,ps1.z.err,CONTEXT,Z-SPEC,STRING
0,5.193125e-29,2.085673e-29,1.097513e-28,5.452987e-31,1.115556e-28,3.765381e-31,1.774660e-28,8.862126e-31,8.117420e-29,5.452048e-31,31,0.03185,2007aj
1,1.349914e-28,1.703768e-29,5.830774e-28,1.421182e-30,9.266348e-28,1.188944e-30,1.525183e-27,3.133268e-30,1.706261e-27,2.442346e-30,31,0.03509,2002G
2,2.644002e-27,3.019760e-29,1.422786e-26,9.195643e-30,2.579356e-26,6.102070e-30,4.004981e-26,1.521644e-29,4.381120e-26,1.152291e-29,31,0.02824,ASASSN-15la


### Run the fit

It will take time for the first time you execute it, as it has to create the template files...<br>
But then, if you relaunch it, will be fast (unless 'update_init' attribute is True).<br>

There are a few options accepted:
- 'filters': allow the user to choose which ones of the available filters in the input data to use for the SED fitting (basically changing the 'CONTEXT' value). It is associated to the parameter 'contextid'. Default is None, meaning that every filters in the input data are used.
- 'contextid': if 'filters' (else than None) is given, allow the user to choose on which row(s) of the input data to change the 'CONTEXT'. Default is None, meaning that every rows will be changed.
- 'dirout': allow the user to change the output directory. Default is None, meaning that it is not changed.
- 'configfile': allow the user to change the configuration file, and so the whole configuration parameters. Default is None, meaning that it is not changed.
- 'catinfile': if 'configfile' is given (else than None), allow the user to change the input data with a file directory. Default is None, meaning that it is not changed.
- 'onwhat': let the user choose on which of the three LePhare available template kinds to run the fit. Default is the use of the three ones ["star","qso","gal"].
- 'gallib': choice of the library used for galaxy templates. Default is "BC03".
- 'update_init': execute the SED fitting initialization, even if the templates already exist. Default is False.

In [9]:
lp.run()

gal:  True star:  True qso:  True ; gallib:  BC03
calling set_zphotlib
filter_file:  sdssu_ps1g_ps1r_ps1i_ps1z.filt
configfile: /Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//config
['Id000000002.spec', 'Id000000000.spec', 'Id000000001.spec']
moved to
['/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000002.spec', '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000000.spec', '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000001.spec']


{'config': '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//config',
 'catin': '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//data.csv',
 'catout': '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//catout',
 'spec': ['/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000002.spec',
  '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000000.spec',
  '/Users/mart1/Desktop/Libraries/lephare/lepharework/pylephare/test_ztf_meeting//Id000000001.spec']}